In [1]:
import pickle
import seaborn as sns

In [2]:
# load pickle files before append new path to the sys.path for loanding custom modules
loaded_model = pickle.load(open(r'N:\10. Executive\Original\3. 2019\Gender Classifier\gender_classifier.pkl', 'rb'))
loaded_vectorizer = pickle.load(open(r'N:\10. Executive\Original\3. 2019\Gender Classifier\vectorizer.pkl', 'rb'))

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import sqlalchemy
import file_utils
import db_utils
import config

#set working directory
os.chdir(config.working_dir)

ImportError: DLL load failed: The specified module could not be found.

In [5]:
# db credentials authentication

In [ ]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

### Import Data

In [7]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_with_names = pd.read_sql_query('''
select a.bra_code, a.cus_num,
(case when cus_name like '%XXX%' then sec_cus_sho_name else cus_name end) cus_name
from cs_retail_loan_cust a, stg.src_customer_extd@exadata_new b
where a.bra_code=b.bra_code and a.cus_num=b.cus_num
''', con)
con.close()

In [8]:
df_loan_data_basic_var = pd.read_csv('df_loan_data_basic_var.csv').merge(
        cs_retail_loan_cust_with_names, on=['BRA_CODE','CUS_NUM'], how='left')

In [9]:
# Convert column types
df_loan_data_basic_var['DATE_OPEN'] = pd.to_datetime(df_loan_data_basic_var['DATE_OPEN'])
df_loan_data_basic_var['BIR_DATE'] = pd.to_datetime(df_loan_data_basic_var['BIR_DATE'])
df_loan_data_basic_var['CUS_DATE_OPEN'] = pd.to_datetime(df_loan_data_basic_var['CUS_DATE_OPEN'])

In [10]:
# df_loan_data_basic_var[df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO']>1][[
#     'CUS_DATE_OPEN','DATE_OPEN','AIRTIME_NBR_MONTH_ACTIVE_RATIO','AIRTIME_NBR_MONTH_ACTIVE','BANK_MTHS']]

### Derive Variables

In [11]:
df_loan_data_basic_var['CUS_AGE_AT_LOAN'] = \
    df_loan_data_basic_var.apply(lambda x: relativedelta(x['DATE_OPEN'], x['BIR_DATE']).years, axis=1)

In [12]:
df_loan_data_basic_var['BANK_YRS'] = \
    df_loan_data_basic_var.apply(lambda x: relativedelta(x['DATE_OPEN'], x['CUS_DATE_OPEN']).years, axis=1)

In [13]:
df_loan_data_basic_var['BANK_MTHS'] = \
    df_loan_data_basic_var.apply(lambda x: (12*x['BANK_YRS']) + relativedelta(x['DATE_OPEN'], x['CUS_DATE_OPEN']).months, axis=1)

In [14]:
# Ratio denominator should not be greater than 12
df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'] = \
    df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE']/df_loan_data_basic_var['BANK_MTHS'].apply(lambda x: x if x<12 else 12)

df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'] = \
    df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].apply(lambda x: x if x<=1 else 1)

In [15]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'] = \
    df_loan_data_basic_var['NTWRK_OUT_DEGREE']/df_loan_data_basic_var['NTWRK_IN_DEGREE']

df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'] = \
    df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].replace(np.inf,np.nan).fillna(0)

In [16]:
# df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'] = ((df_loan_data_basic_var['POS_TRA_COUNT_0_5'] + 
#                                                               df_loan_data_basic_var['POS_TRA_COUNT_21_23']
#                                                              )/ df_loan_data_basic_var['POS_TRA_COUNT'])

# df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'] = df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'].replace(np.inf,np.nan).fillna(0)
# df_loan_data_basic_var.loc[df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO']>=0.2,'IS_POS_NIGHT_SPENDER']=1
# df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'] = df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'].fillna(0)

In [17]:
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'] = \
                                    (df_loan_data_basic_var['POS_TRA_DAY_COUNT_0_5']+   \
                                     df_loan_data_basic_var['POS_TRA_DAY_COUNT_21_23'])/ \
                                    df_loan_data_basic_var['BANK_MTHS']
        
df_loan_data_basic_var.loc[df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO']>=0.2,'IS_POS_NIGHT_SPENDER']=1
df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'] = df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'].fillna(0)

In [18]:
df_loan_data_basic_var['IB_MB_GTW_COUNT'] = df_loan_data_basic_var['IB_CNT'] + \
                                            df_loan_data_basic_var['MB_CNT'] + \
                                            df_loan_data_basic_var['GTWORLD_CNT']
        
df_loan_data_basic_var['USE_IB_MB_GTW'] = (df_loan_data_basic_var['IB_MB_GTW_COUNT']>0).astype(int)

In [19]:
df_loan_data_basic_var['HAS_IJEBU_LEDGERS']= (df_loan_data_basic_var['NBR_DISTINCT_IJEBU_LEDGERS']>0).astype(int)

In [20]:
df_loan_data_basic_var['HAS_EVER_LOAN']= (df_loan_data_basic_var['NBR_LOANS']>0).astype(int)

In [21]:
df_loan_data_basic_var['HAS_DOM_ACCS']= (df_loan_data_basic_var['NBR_DOM_ACCS']>0).astype(int)

In [22]:
df_loan_data_basic_var['USE_MB_AND_GTW']= \
    ((df_loan_data_basic_var['MB_CNT']>0) & (df_loan_data_basic_var['GTWORLD_CNT']>0)).astype(int)

In [23]:
df_loan_data_basic_var['USE_POS_OR_GAPS']= \
    ((df_loan_data_basic_var['POS_CNT']>0) | (df_loan_data_basic_var['GAPS_CNT']>0)).astype(int)

In [24]:
df_loan_data_basic_var['USE_ATM_ONLY']= \
(((df_loan_data_basic_var['ATM_ROU_CNT']+df_loan_data_basic_var['ATM_ON_US_CNT']==0)>0) & \
 (df_loan_data_basic_var['GTWORLD_CNT']==0) & \
 (df_loan_data_basic_var['IB_CNT']==0) & \
 (df_loan_data_basic_var['POS_WDR_CNT']==0) & \
 (df_loan_data_basic_var['USSD_CNT']==0)).astype(int)

In [25]:
df_loan_data_basic_var['USE_USSD_ONLY']= \
((df_loan_data_basic_var['USSD_CNT']>0) & \
 (df_loan_data_basic_var['ATM_ROU_CNT']==0) & \
 (df_loan_data_basic_var['ATM_ON_US_CNT']==0) & \
 (df_loan_data_basic_var['GTWORLD_CNT']==0) & \
 (df_loan_data_basic_var['IB_CNT']==0) & \
 (df_loan_data_basic_var['POS_WDR_CNT']==0)).astype(int)

In [26]:
df_loan_data_basic_var['HAS_BAD_NEIGHBORS'] = (df_loan_data_basic_var['NBR_BAD_NEIGHBORS']>0).astype(int)

In [27]:
df_loan_data_basic_var['HAS_CONSISTENT_LODGE']= (df_loan_data_basic_var['MONTH_ACTIVE_RATIO']>=0.9).astype(int)

In [28]:
df_loan_data_basic_var['DOES_IN_BRANCH']= (df_loan_data_basic_var['CASH_WDR_CNT']>0).astype(int)

In [29]:
df_loan_data_basic_var['ISSUES_CHEQUE']= (df_loan_data_basic_var['CHEQUES_ISSUED']>0).astype(int)

In [30]:
df_loan_data_basic_var['REACTIVATED_OR_NEW_IN_12_MTHS'] = \
    ((df_loan_data_basic_var['HAS_REACTIVATED_12_MTHS']>0) | (df_loan_data_basic_var['BANK_YRS']==0)).astype(int)

In [31]:
df_loan_data_basic_var['ISSUES_CHEQUE']= (df_loan_data_basic_var['CHEQUES_ISSUED']>0).astype(int)

In [32]:
df_loan_data_basic_var[df_loan_data_basic_var.USE_ATM_ONLY>0].IS_BAD.value_counts()

0.0    1330
1.0     105
Name: IS_BAD, dtype: int64

In [33]:
# df_loan_data_basic_var[df_loan_data_basic_var['HAS_BAD_NEIGHBORS']==1].IS_BAD.value_counts()

In [34]:
# df_loan_data_basic_var[df_loan_data_basic_var['REACTIVATED_OR_NEW_IN_12_MTHS']==1].IS_BAD.value_counts()

In [35]:
# df_loan_data_basic_var[df_loan_data_basic_var['IS_PEP']==1].IS_BAD.value_counts()

In [36]:
# df_loan_data_basic_var[df_loan_data_basic_var['BANK_YRS']==0]

In [37]:
df_loan_data_basic_var[df_loan_data_basic_var.GAPS_CNT>0].IS_BAD

515       0.0
631       0.0
737       0.0
825       0.0
1275      0.0
1481      0.0
3257      0.0
3317      0.0
3380      0.0
3472      0.0
3512      0.0
3718      0.0
3828      0.0
4166      0.0
4561      0.0
4719      0.0
4722      0.0
4770      0.0
4882      0.0
5281      0.0
5294      0.0
5425      0.0
5691      0.0
5941      0.0
6136      0.0
6142      0.0
6259      0.0
6308      0.0
6338      0.0
6372      0.0
         ... 
172354    0.0
172592    0.0
172700    0.0
172910    0.0
172981    0.0
173166    0.0
173715    0.0
173781    0.0
174004    0.0
174011    0.0
174326    0.0
174355    0.0
174377    0.0
174430    0.0
174475    0.0
174513    0.0
174574    0.0
175196    0.0
175209    0.0
175260    0.0
175336    0.0
175380    0.0
175509    0.0
175602    0.0
175607    0.0
175640    0.0
176079    0.0
176085    0.0
176104    0.0
176169    0.0
Name: IS_BAD, Length: 1290, dtype: float64

In [38]:
df_loan_data_basic_var[df_loan_data_basic_var.POS_CNT>0].IS_BAD

1089      0.0
1690      0.0
2095      0.0
2147      0.0
2171      0.0
2318      0.0
2415      0.0
3266      0.0
3738      0.0
3802      0.0
5726      0.0
6221      0.0
6488      0.0
9351      0.0
10842     0.0
11016     0.0
11297     0.0
11712     0.0
11741     0.0
11783     0.0
13152     0.0
15213     0.0
15809     0.0
17029     0.0
17722     0.0
18812     0.0
18969     0.0
19022     0.0
19378     0.0
19944     0.0
         ... 
148798    0.0
148921    0.0
149005    0.0
149153    0.0
149374    0.0
155674    0.0
156268    0.0
157108    0.0
157389    0.0
159402    0.0
161323    0.0
162323    0.0
162942    0.0
163089    0.0
163154    0.0
164977    0.0
167375    0.0
170010    0.0
170406    0.0
170639    0.0
170948    0.0
171151    0.0
171346    0.0
171481    0.0
172729    0.0
173446    0.0
174194    0.0
175140    0.0
175160    0.0
176168    0.0
Name: IS_BAD, Length: 270, dtype: float64

In [39]:
df_loan_data_basic_var[df_loan_data_basic_var.HAS_BIZ_ACC>0].IS_BAD.value_counts()

0.0    6417
1.0     210
Name: IS_BAD, dtype: int64

### Exploration

In [40]:
df_loan_data_basic_var.COV_LODGEMENT_CNT.describe()

count    176476.000000
mean          0.537578
std           0.233595
min           0.000000
25%           0.384294
50%           0.504469
75%           0.648389
max           3.316625
Name: COV_LODGEMENT_CNT, dtype: float64

In [41]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].describe()
df_loan_data_basic_var['NTWRK_DEGREE'].describe()
df_loan_data_basic_var['AIRTIME_AVG_PER_MONTH_COUNT'].describe()
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'].describe()
df_loan_data_basic_var['SUM_LODGEMENT_AMOUNT'].describe()
print df_loan_data_basic_var['AVG_TURNOVER_AMOUNT'].describe()
print df_loan_data_basic_var['AVG_LODGEMENT_AMOUNT'].describe()
# df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].describe()

count    1.764760e+05
mean     4.244441e+05
std      1.486584e+06
min      0.000000e+00
25%      5.667991e+04
50%      1.242430e+05
75%      3.200492e+05
max      1.705020e+08
Name: AVG_TURNOVER_AMOUNT, dtype: float64
count    1.764760e+05
mean     4.233995e+05
std      1.487154e+06
min      9.166667e+02
25%      5.713403e+04
50%      1.245724e+05
75%      3.192337e+05
max      1.705622e+08
Name: AVG_LODGEMENT_AMOUNT, dtype: float64


In [42]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].describe()
df_loan_data_basic_var['NTWRK_DEGREE'].describe()
df_loan_data_basic_var['AIRTIME_AVG_PER_MONTH_COUNT'].describe()
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'].describe()
df_loan_data_basic_var['SUM_LODGEMENT_AMOUNT'].describe()
df_loan_data_basic_var['AVG_TURNOVER_AMOUNT'].describe()
df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].describe()

count    176476.000000
mean          0.758249
std           0.330823
min           0.000000
25%           0.600000
50%           0.916667
75%           1.000000
max           1.000000
Name: AIRTIME_NBR_MONTH_ACTIVE_RATIO, dtype: float64

### Binning

In [43]:
bin_args = \
[
 {'col':'CUS_AGE_AT_LOAN', 'bins':[-1,17,29,39,44,999], 'right':True, 'labels':None, 'precision':5},
 {'col':'BANK_YRS', 'bins':[-1,3,5,7,99], 'right':True, 'labels':None, 'precision':5},
 
 {'col':'NBR_DISTINCT_DEP', 'bins':[-1,1,5,99], 'right':True, 'labels':None, 'precision':5},
 {'col':'AIRTIME_NBR_MONTH_ACTIVE_RATIO', 'bins':[-1,0.5,0.8,0.9,1], 'right':True, 'labels':None, 'precision':5},
 {'col':'AIRTIME_AVG_PER_MONTH_COUNT', 'bins':[-1,2,5,10,20,99999], 'right':True, 'labels':None, 'precision':5},
 
 {'col':'NTWRK_OUT_IN_DEGREE_RATIO', 'bins':[-1,0.1,1,2,99999], 'right':True, 'labels':None, 'precision':5},
 {'col':'NTWRK_DEGREE', 'bins':[-1,3,11,30,99999], 'right':True, 'labels':None, 'precision':5},
    
 {'col':'SUM_LODGEMENT_AMOUNT', 'bins':[-1,50000,250000,750000,2000000,999999999], 'right':True, 'labels':None, 'precision':5},
 {'col':'AVG_TURNOVER_AMOUNT', 'bins':[-1,3000,6000,10000,99999999], 'right':True, 'labels':None, 'precision':5},
 {'col':'AVG_LODGEMENT_AMOUNT', 'bins':[-1,3000,6000,10000,99999999], 'right':True, 'labels':None, 'precision':5},
    
 {'col':'COV_LODGEMENT_CNT', 'bins':[-1,0.3,0.5,0.7,1,99999999], 'right':True, 'labels':None, 'precision':5}
]

In [44]:
# Save binning arguments
pickle.dump(bin_args, open('bin_args.sav', 'wb'))

In [45]:
def bin_retail_loan_dataset(bin_args, df):
    for arg in bin_args:
        suffix_for_new_col = '_BIN'
        cur_col_name = arg['col']
        cur_arg_bins = arg['bins']
        cur_arg_right = arg['right']
        cur_arg_precision = arg['precision']
        print 'Running for: ', cur_col_name

        cur_col_to_bin = df[cur_col_name]

        df[cur_col_name + suffix_for_new_col] = pd.Series(pd.cut(cur_col_to_bin, bins=cur_arg_bins,
                                                                                 right=cur_arg_right,
                                                                                 precision=cur_arg_precision))
    return df

In [46]:
df_loan_data_basic_var = bin_retail_loan_dataset(bin_args, df_loan_data_basic_var)

Running for:  CUS_AGE_AT_LOAN
Running for:  BANK_YRS
Running for:  NBR_DISTINCT_DEP
Running for:  AIRTIME_NBR_MONTH_ACTIVE_RATIO
Running for:  AIRTIME_AVG_PER_MONTH_COUNT
Running for:  NTWRK_OUT_IN_DEGREE_RATIO
Running for:  NTWRK_DEGREE
Running for:  SUM_LODGEMENT_AMOUNT
Running for:  AVG_TURNOVER_AMOUNT
Running for:  AVG_LODGEMENT_AMOUNT
Running for:  COV_LODGEMENT_CNT


### Fill missing gender values by Gender Prediction Model

In [47]:
def transform_name(name):
    return loaded_vectorizer.transform([name.upper().strip()])
def genderpredictor(name):
    return loaded_model.predict(transform_name(name))

In [48]:
# predict gender that is null
df_loan_data_basic_var['CUS_GENDER'] = df_loan_data_basic_var.apply(
    lambda x: genderpredictor(x.CUS_NAME)[0] if not x.CUS_GENDER in ['M','F'] else x.CUS_GENDER, axis=1)

In [49]:
pd.DataFrame(df_loan_data_basic_var.columns).to_excel('retail_loan_cust_columns.xlsx', index=False)

### Feature Selection

In [56]:
cols_selected = ['CUS_AGE_AT_LOAN_BIN', 'BANK_YRS_BIN', 'NBR_DISTINCT_DEP_BIN', 'AIRTIME_NBR_MONTH_ACTIVE_RATIO_BIN',
                 'AIRTIME_AVG_PER_MONTH_COUNT_BIN', 'NTWRK_OUT_IN_DEGREE_RATIO_BIN', 'NTWRK_DEGREE_BIN', 'IS_BETTOR',
                 'IS_RELIGIOUS', 'IS_VISA', 'IS_PAY_UTILITIES', 'IS_POS_NIGHT_SPENDER', 'CUS_GENDER', 'HAS_REACTIVATED',
                 'AIRTIME_DO_THIRD_PARTY', 'HAS_IJEBU_LEDGERS', 'HAS_EVER_LOAN', 'USE_IB_MB_GTW',
                 'IS_PLATINUM_PLUS_HNI', 'AVG_TURNOVER_AMOUNT_BIN','HAS_BIZ_ACC','REACTIVATED_OR_NEW_IN_12_MTHS',
                 'ISSUES_CHEQUE','DOES_IN_BRANCH','HAS_CONSISTENT_LODGE','HAS_BAD_NEIGHBORS',
                 'HAS_DOM_ACCS','USE_MB_AND_GTW','USE_POS_OR_GAPS','USE_ATM_ONLY','COV_LODGEMENT_CNT_BIN','STATE','PROF_CODE']

In [57]:
df_loan_data_final_set = df_loan_data_basic_var[cols_selected+['BRA_CODE', 'CUS_NUM','IS_BAD']]

In [58]:
pickle.dump(cols_selected, open('cols_selected.sav', 'wb'))

### Check missing values

In [59]:
df_loan_data_final_set.isnull().sum()

CUS_AGE_AT_LOAN_BIN                     0
BANK_YRS_BIN                            0
NBR_DISTINCT_DEP_BIN                    0
AIRTIME_NBR_MONTH_ACTIVE_RATIO_BIN      0
AIRTIME_AVG_PER_MONTH_COUNT_BIN         0
NTWRK_OUT_IN_DEGREE_RATIO_BIN           0
NTWRK_DEGREE_BIN                        0
IS_BETTOR                               0
IS_RELIGIOUS                            0
IS_VISA                                 0
IS_PAY_UTILITIES                        0
IS_POS_NIGHT_SPENDER                    0
CUS_GENDER                              0
HAS_REACTIVATED                         0
AIRTIME_DO_THIRD_PARTY                  0
HAS_IJEBU_LEDGERS                       0
HAS_EVER_LOAN                           0
USE_IB_MB_GTW                           0
IS_PLATINUM_PLUS_HNI                    0
AVG_TURNOVER_AMOUNT_BIN                 4
HAS_BIZ_ACC                             0
REACTIVATED_OR_NEW_IN_12_MTHS           0
ISSUES_CHEQUE                           0
DOES_IN_BRANCH                    

### Write output

In [60]:
df_loan_data_final_set.to_csv('df_loan_data_final_set.csv', index=False)

In [61]:
df_loan_data_final_set

,CUS_AGE_AT_LOAN_BIN,BANK_YRS_BIN,NBR_DISTINCT_DEP_BIN,AIRTIME_NBR_MONTH_ACTIVE_RATIO_BIN,AIRTIME_AVG_PER_MONTH_COUNT_BIN,NTWRK_OUT_IN_DEGREE_RATIO_BIN,NTWRK_DEGREE_BIN,IS_BETTOR,IS_RELIGIOUS,IS_VISA,...,HAS_DOM_ACCS,USE_MB_AND_GTW,USE_POS_OR_GAPS,USE_ATM_ONLY,COV_LODGEMENT_CNT_BIN,STATE,PROF_CODE,BRA_CODE,CUS_NUM,IS_BAD
0,"(44, 999]","(7, 99]","(1, 5]","(0.9, 1.0]","(-1, 2]","(2.0, 99999.0]","(30, 99999]",0.0,1.0,0.0,...,1,0,0,0,"(0.3, 0.5]",LA,81.0,201,100084,0.0
1,"(44, 999]","(7, 99]","(1, 5]","(0.9, 1.0]","(2, 5]","(0.1, 1.0]","(30, 99999]",0.0,0.0,0.0,...,1,0,0,0,"(0.5, 0.7]",LA,81.0,201,100116,0.0
2,"(44, 999]","(7, 99]","(-1, 1]","(0.9, 1.0]","(2, 5]","(2.0, 99999.0]","(30, 99999]",0.0,1.0,0.0,...,1,1,0,0,"(1.0, 99999999.0]",LA,2.0,201,100160,0.0
3,"(44, 999]","(7, 99]","(1, 5]","(0.9, 1.0]","(5, 10]","(2.0, 99999.0]","(30, 99999]",0.0,0.0,0.0,...,1,0,0,0,"(0.3, 0.5]",LA,104.0,201,100166,0.0
4,"(44, 999]","(7, 99]","(1, 5]","(-1.0, 0.5]","(-1, 2]","(2.0, 99999.0]","(30, 99999]",0.0,0.0,0.0,...,1,0,0,0,"(0.3, 0.5]",LA,104.0,201,100347,0.0
5,"(44, 999]","(7, 99]","(1, 5]","(0.9, 1.0]","(10, 20]","(1.0, 2.0]","(11, 30]",0.0,1.0,0.0,...,0,0,0,0,"(0.3, 0.5]",LA,2.0,201,100401,0.0
6,"(44, 999]","(7, 99]","(1, 5]","(0.9, 1.0]","(10, 20]","(2.0, 99999.0]","(30, 99999]",0.0,1.0,0.0,...,1,0,0,0,"(-1.0, 0.3]",LA,10.0,201,100409,0.0
7,"(17, 29]","(7, 99]","(1, 5]","(-1.0, 0.5]","(-1, 2]","(2.0, 99999.0]","(30, 99999]",0.0,1.0,0.0,...,1,1,0,0,"(-1.0, 0.3]",LA,99.0,201,100470,0.0
8,"(44, 999]","(7, 99]","(1, 5]","(-1.0, 0.5]","(-1, 2]","(2.0, 99999.0]","(30, 99999]",0.0,1.0,0.0,...,1,0,0,0,"(0.3, 0.5]",LA,87.0,201,100497,0.0
9,"(44, 999]","(7, 99]","(-1, 1]","(0.9, 1.0]","(2, 5]","(1.0, 2.0]","(11, 30]",0.0,0.0,0.0,...,0,0,0,0,"(0.5, 0.7]",LA,16.0,201,100557,0.0


In [ ]:


sns.catplot(x="sex", y="total_bill",
            hue="is_bad", col="time",
            data=df_loan_data_final_set, kind="bar",
            height=4, aspect=.7)

In [62]:
sns.catplot(hue="is_bad", col="time",
                data=df_loan_data_final_set, kind="bar",
                height=4, aspect=.7)

NameError: name 'sns' is not defined